load the data and split into train val test when the dataset is loaded as tensors (thanks to pytorch)
Do the training here by calling train() and defining optimizer,model, etc. so we keep the output
We can also test the model here